In [1]:
import numpy as np
import re
import string
import unicodedata
from unicodedata import normalize
import sys
import indicnlp
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.utils import plot_model
import numpy as np
from sklearn.model_selection import train_test_split
import gensim

In [2]:
def load_doc(filename):
    file = open(filename , mode ='rt' , encoding = 'utf-8')
    text = file.read()
    file.close()
    return text

In [3]:
def to_pairs(text):
    lines = text.strip().split('\n') #1d list of strings
    eng_hin_pairs = [line.split('\t') for line in lines] #list
    eng_hin_pairs = np.delete(eng_hin_pairs , [0,1,2,3,4] , axis = 1)
#     eng_hin_pairs = np.delete(eng_hin_pairs , 0 , axis = 1)
    eng_hin_pairs = eng_hin_pairs.tolist()
    return eng_hin_pairs   

In [4]:
doc = load_doc('hindi-visual-genome-train.txt')
print(doc)

hin_eng_pairs = to_pairs(doc)

english_sentences = []
hindi_sentences = []
for i in range(len(hin_eng_pairs)):
    english_sentences.append(hin_eng_pairs[i][0])
    hindi_sentences.append(hin_eng_pairs[i][1])
len(hin_eng_pairs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



28930

In [5]:
hin_eng_pairs

[['it is an indoor scene', 'यह एक इनडोर दृश्य है'],
 ['Computer screens turned on', 'कंप्यूटर स्क्रीन चालू'],
 ['man has short hair', 'आदमी के छोटे बाल हैं'],
 ["photo album open on an adult's lap", 'एक वयस्क की गोद में फोटो एल्बम खुला'],
 ['there is a group of girls beside the black car',
  'काली कार के पास लड़कियों का एक समूह है'],
 ['Child in a stroller', 'एक घुमक्कड़ में बच्चा'],
 ['Tall metal lightpost', 'लंबा धातु प्रकाश पोस्ट'],
 ['wall is painted white', 'दीवार सफेद रंग की है'],
 ['there are several pictures on the wall', 'दीवार पर कई चित्र हैं'],
 ['woman facing the ocean', 'सागर का सामना कर रही महिला'],
 ['this is an office layout', 'यह एक कार्यालय लेआउट है'],
 ['four metallic chairs', 'चार धातु की कुर्सियाँ'],
 ['Clutter is on a table', 'क्लटर एक मेज पर है'],
 ['a white microwave oven', 'एक सफेद माइक्रोवेव ओवन'],
 ['White SUV driving through intersection',
  'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग'],
 ['Person crossing street with umbrella', 'छतरी के साथ सड़क पार करते व्य

In [6]:
len(hin_eng_pairs)

28930

In [7]:
print(type(hin_eng_pairs))

<class 'list'>


In [8]:
hindi_sentences

['यह एक इनडोर दृश्य है',
 'कंप्यूटर स्क्रीन चालू',
 'आदमी के छोटे बाल हैं',
 'एक वयस्क की गोद में फोटो एल्बम खुला',
 'काली कार के पास लड़कियों का एक समूह है',
 'एक घुमक्कड़ में बच्चा',
 'लंबा धातु प्रकाश पोस्ट',
 'दीवार सफेद रंग की है',
 'दीवार पर कई चित्र हैं',
 'सागर का सामना कर रही महिला',
 'यह एक कार्यालय लेआउट है',
 'चार धातु की कुर्सियाँ',
 'क्लटर एक मेज पर है',
 'एक सफेद माइक्रोवेव ओवन',
 'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग',
 'छतरी के साथ सड़क पार करते व्यक्ति',
 'इमारत पर झुकाव में आदमी ग्रे पैंट में',
 'भवन पर खिड़की',
 'कारों के बीच में खड़ा एक आदमी',
 'दीवार पर चित्रकला',
 'कुर्सी का पैर',
 'एक नाव की खिड़की',
 'कैबिनेट लकड़ी है',
 'एक आदमी एक किताब पढ़ रहा है',
 'पानी में तैरता हुआ व्यक्ति',
 'चमकती छाया के साथ दीपक',
 'प्लेट पर फल',
 'एक चीनी मिट्टी के बरतन सफ़ेद प्लेट',
 'काला खेल नियंत्रक',
 'कार की डिक्की में कुछ डालने वाला व्यक्ति।',
 'एक सड़क के साथ ग्रीन कार पार्क किया',
 'गली पर कार',
 'निर्माण कर्मचारी कवरलेस पहनते हैं',
 'पीले दरवाजे के साथ एक काली कार',
 '

In [9]:
def clean_english_data(lines):
    re_print = re.compile('[^%s]' %re.escape(string.printable))
    table = str.maketrans('' , '' , string.punctuation)
    clean_eng_lines = []
    for line in lines:
        line = normalize('NFD' , line).encode('ascii' , 'ignore')
        line = line.decode('UTF-8')
        line = line.split()
        line = [word.lower() for word in line]
        line = [word.translate(table) for word in line]
        line = [re_print.sub('' , w) for w in line]
        line = [word for word in line if word.isalpha()]
        line = ' '.join(line)
        clean_eng_lines.append(line)
    return clean_eng_lines    

In [10]:
clean_eng_lines = clean_english_data(english_sentences)

# print(english_sentences)
print(english_sentences[3])
print(clean_eng_lines[3])

photo album open on an adult's lap
photo album open on an adults lap


In [11]:
INDIC_NLP_LIB_HOME=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\anoopkunchukuttan-indic_nlp_library-eccde81"
INDIC_NLP_RESOURCES=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [12]:
def clean_text(line):
    text = line
    text = text.replace(u',' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u'(' ,'')
    text = text.replace(u')' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u':' ,'')
    text = text.replace(u"'" ,'')
    text = text.replace(u"'" ,'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text = re.sub('[a-zA-Z]' , '' , text)
    text = re.sub('[0-9+\-*/.%&!]' , '' , text)
    return text

In [13]:
def clean_hindi_data(lines):
    clean_hin_lines = []
    for line in lines:
        remove_nuktas = True
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer("hi" , remove_nuktas = True )
        line = clean_text(line)
        tokens = list()
        for t in indic_tokenize.trivial_tokenize(line):
            tokens.append(t)
        line = tokens
        line = [ word.lower() for word in line]
        line = [word for word in line if not re.search(r'\d', word)]
        line = ' '.join(line)
        clean_hin_lines.append(line)
    return clean_hin_lines  

In [14]:
clean_hindi_lines = clean_hindi_data(hindi_sentences)
print(hindi_sentences[133])
print(clean_hindi_lines[133])

तार से जुड़ा तार
तार से जुड़ा तार


In [15]:
len(clean_eng_lines)

28930

In [16]:
len(clean_hindi_lines)

28930

In [17]:
input_train , input_val, target_train, target_val = train_test_split(clean_eng_lines , clean_hindi_lines , test_size = 0.2, random_state = 42)

### Adding SOS and EOS ,  PREPARING INPUTS FOR ENCODER AND DECODER

In [18]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

In [19]:
#input sentences
input_sentences = input_train

#output sentences
for line in target_train:
    line = line + ' <eos>'
    output_sentences.append(line)

#output sentence input

for line in target_train:
    line = '<sos> ' + line
    output_sentences_inputs.append(line)


In [20]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 23144
num samples output: 23144
num samples output input: 23144


In [21]:
BATCH_SIZE = 64
EPOCHS = 10
LSTM_NODES = 256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [22]:
# for english vocabulary

In [23]:
eng_lang_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
eng_lang_tokenizer.fit_on_texts(clean_eng_lines)
input_integer_seq = eng_lang_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = eng_lang_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 5550
Length of longest sentence in input: 24


In [26]:
word2idx_inputs

{'a': 1,
 'the': 2,
 'on': 3,
 'of': 4,
 'is': 5,
 'in': 6,
 'white': 7,
 'man': 8,
 'and': 9,
 'black': 10,
 'with': 11,
 'red': 12,
 'blue': 13,
 'this': 14,
 'woman': 15,
 'green': 16,
 'brown': 17,
 'person': 18,
 'two': 19,
 'wearing': 20,
 'yellow': 21,
 'sign': 22,
 'train': 23,
 'sky': 24,
 'wall': 25,
 'water': 26,
 'are': 27,
 'building': 28,
 'an': 29,
 'table': 30,
 'people': 31,
 'standing': 32,
 'street': 33,
 'large': 34,
 'tennis': 35,
 'head': 36,
 'sitting': 37,
 'part': 38,
 'clock': 39,
 'bus': 40,
 'dog': 41,
 'small': 42,
 'light': 43,
 'plate': 44,
 'holding': 45,
 'to': 46,
 'cat': 47,
 'grass': 48,
 'shirt': 49,
 'window': 50,
 'has': 51,
 'at': 52,
 'orange': 53,
 'side': 54,
 'ground': 55,
 'plane': 56,
 'bear': 57,
 'boy': 58,
 'car': 59,
 'top': 60,
 'giraffe': 61,
 'girl': 62,
 'playing': 63,
 'road': 64,
 'snow': 65,
 'baseball': 66,
 'field': 67,
 'horse': 68,
 'bench': 69,
 'air': 70,
 'silver': 71,
 'elephant': 72,
 'front': 73,
 'walking': 74,
 'pizza

In [24]:
# for hindi vocabulary
hindi = []
for i in clean_hindi_lines:
    i = '<sos> ' + i + ' <eos>'
    hindi.append(i)
len(hindi)

hin_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
hin_lang_tokenizer.fit_on_texts(hindi)
output_integer_seq = hin_lang_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = hin_lang_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = hin_lang_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 7013
Length of longest sentence in the output: 33


In [27]:
word2idx_outputs

{'<OOV>': 1,
 '<sos>': 2,
 '<eos>': 3,
 'एक': 4,
 'पर': 5,
 'है': 6,
 'के': 7,
 'में': 8,
 'का': 9,
 'सफेद': 10,
 'की': 11,
 'आदमी': 12,
 'और': 13,
 'लाल': 14,
 'साथ': 15,
 'यह': 16,
 'व्यक्ति': 17,
 'हैं': 18,
 'रंग': 19,
 'महिला': 20,
 'रहा': 21,
 'दो': 22,
 'सड़क': 23,
 'से': 24,
 'हुए': 25,
 'काले': 26,
 'काला': 27,
 'ट्रेन': 28,
 'दीवार': 29,
 'पानी': 30,
 'पहने': 31,
 'आकाश': 32,
 'नीला': 33,
 'टेनिस': 34,
 'बस': 35,
 'बिल्ली': 36,
 'हिस्सा': 37,
 'भूरे': 38,
 'घड़ी': 39,
 'सिर': 40,
 'लोग': 41,
 'घास': 42,
 'हुआ': 43,
 'वाला': 44,
 'प्लेट': 45,
 'काली': 46,
 'नीले': 47,
 'हाथी': 48,
 'को': 49,
 'इमारत': 50,
 'मेज': 51,
 'लकड़ी': 52,
 'रही': 53,
 'हरे': 54,
 'लड़की': 55,
 'नारंगी': 56,
 'खिड़की': 57,
 'कर': 58,
 'जिराफ': 59,
 'खेल': 60,
 'कुत्ता': 61,
 'कार': 62,
 'जमीन': 63,
 'शर्ट': 64,
 'बर्फ': 65,
 'पीले': 66,
 'खड़ा': 67,
 'नीली': 68,
 'बेंच': 69,
 'लड़का': 70,
 'विमान': 71,
 'पेड़': 72,
 'बेसबॉल': 73,
 'हवा': 74,
 'हरी': 75,
 'किनारे': 76,
 'पीला': 77,
 'ग्रे': 78,
 'टोपी':

In [25]:
# pad encoder inputs and decoder inputs
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen = max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

encoder_input_sequences.shape: (23144, 24)
encoder_input_sequences[172]: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34 17 41]
decoder_input_sequences.shape: (23144, 33)
decoder_input_sequences[172]: [ 2 85 88 61  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0]


### word embeddings

In [28]:
# english embeddings
english = [line.split() for line in clean_eng_lines]

In [29]:
w2v_model = gensim.models.Word2Vec(english , vector_size = 100 , min_count = 1 , window = 2)

In [30]:
w2v_model.train(english , epochs = 50 , total_examples = len(clean_eng_lines))

(4558405, 7147450)

In [31]:
w2v_model.wv.get_vector('like')

array([-0.7444832 ,  1.5174689 , -0.7493498 , -0.02083266, -0.42761415,
       -0.094271  , -0.8511561 ,  0.63892955, -0.20852706,  0.2221497 ,
       -0.29043302,  1.3107636 , -0.6259568 , -0.00717992, -0.35501027,
       -0.58973706,  0.67967254, -1.1222788 , -0.8047087 ,  0.19160202,
        0.45325467,  0.09031003,  0.83277935,  1.022939  , -0.56163675,
       -0.66612935, -0.0702213 , -0.65509564, -0.16416882,  0.93489826,
        1.0321059 , -0.27677074, -1.2082309 , -0.243897  , -0.02980584,
        0.5752531 ,  0.3994933 , -0.19885518,  1.3598133 , -1.349967  ,
       -0.65962327,  0.14425589,  0.68319684,  0.24990605, -0.23707253,
       -0.11001286, -1.0935981 ,  0.0714547 , -0.44840232, -0.14673825,
       -0.15507206, -0.6000024 , -0.2099516 , -0.2925382 , -0.41125476,
        0.6580849 ,  1.0251323 , -0.52222514, -0.5711514 , -0.22152826,
        0.22730201, -0.92945874,  1.068398  ,  1.1498948 , -1.5136915 ,
        0.34093434,  0.30780193,  0.28607315,  0.80126274, -0.50

In [34]:
embeddings_dictionary = {}
for word in eng_lang_tokenizer.word_index:
    try:
        embeddings_dictionary[word] = w2v_model.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary))
# embeddings_dictionary

the no of key-value pairs: 5550


{'a': array([-0.53558475, -0.25552642,  0.4837933 ,  0.44841656,  0.09033091,
        -0.3046522 ,  0.53876936, -0.02008769,  0.04282425,  0.7649327 ,
         0.4740114 , -0.9490094 , -0.47073042,  0.8654011 , -0.5494125 ,
        -0.14005575, -0.6239082 ,  0.5971521 , -0.43326983, -0.9471077 ,
         0.80584913, -0.13410895,  0.68249756,  0.0554007 , -0.9254248 ,
        -0.27885038,  0.09303893, -0.06718511, -0.7942257 ,  0.7751222 ,
        -0.7381243 , -0.15670457, -0.04157142, -0.07479739,  0.878433  ,
         0.63249266,  0.02582964, -0.2853881 ,  0.22869596, -0.19375333,
        -0.46653807, -0.6557937 , -0.60943615, -0.30943447,  0.40303913,
        -0.14223194,  0.14724967,  0.8701591 , -0.05817563, -0.79219353,
        -0.43591943, -0.22986881, -0.848924  , -0.2330188 ,  0.0471343 ,
        -0.53798425, -0.46829063, -0.025465  , -0.84255165, -0.16997337,
         0.25615185,  0.28490373, -0.30754998, -0.37703377,  0.40146992,
        -0.19316712, -0.29970267,  0.07188449,

In [37]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [35]:
# hindi embedding word2vec

w2v_model_h = gensim.models.Word2Vec.load("hi-d100-m2-cbow.model")

In [36]:
embeddings_dictionary_h = {}
for word in hin_lang_tokenizer.word_index:
    try:
        embeddings_dictionary_h[word] = w2v_model_h.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary_h))
# embeddings_dictionary_h

the no of key-value pairs: 6380


{'एक': array([ 1.3761369e+00,  3.1895986e-01,  5.5161637e-01,  3.3454773e+00,
         7.5480199e-01, -4.1311855e+00,  3.6096673e+00,  3.1984558e-03,
         1.1114923e+00,  1.8430928e+00, -1.6482291e+00, -2.6384809e+00,
        -2.3330653e+00, -1.3789964e+00,  2.8163950e+00,  2.2858524e+00,
         2.9680107e+00,  7.1689091e+00,  1.8727484e+00, -3.0235820e+00,
         2.2039354e+00, -1.5322008e+00,  4.8508924e-01,  1.1078984e+00,
        -3.8863792e+00, -1.7010270e-01, -2.9873648e-01, -1.1823559e+00,
        -3.1249235e+00,  2.6842210e-01,  2.3926444e-01, -2.8258526e+00,
        -7.1746130e+00, -8.4229892e-01, -2.8898644e+00,  3.0856481e-01,
        -2.0823562e+00,  5.0447965e-01, -7.3356193e-01,  2.7815120e+00,
         7.8827173e-01,  8.9215755e-01, -1.7951288e+00,  1.1564068e+00,
         5.7838392e-02,  3.4770534e+00, -1.4091858e+00, -2.1184385e+00,
        -3.9627492e+00, -3.0346465e-01, -1.0447567e+00, -1.7222787e+00,
        -1.6216998e+00, -2.7504045e-01,  2.9573914e-02, -2

In [42]:
num_words_h = min(MAX_NUM_WORDS, len(word2idx_outputs) + 1)
embedding_matrix_h = zeros((num_words_h, 100))
for word, index in hin_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary_h.get(word)
    if embedding_vector is not None:
        embedding_matrix_h[index] = embedding_vector

# MODEL

In [46]:
## CREATING THE MODEL

In [47]:
decoder_targets_one_hot = np.zeros((len(input_sentences), max_out_len, num_words_output),dtype='float32')

In [48]:
decoder_targets_one_hot.shape
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

## create the encoder and decoders

In [49]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [50]:
embedding_layer_h = Embedding(num_words_h, EMBEDDING_SIZE, weights=[embedding_matrix_h], input_length=max_out_len)

decoder_inputs_placeholder = Input(shape=(max_out_len,))
decoder_inputs_x = embedding_layer_h(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES , return_sequences = True , return_state = True)

decoder_outputs , _ , _ = decoder_lstm(decoder_inputs_x , initial_state = encoder_states)

In [51]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## compile the model:

In [52]:
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder] , decoder_outputs)

model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [53]:
!pip install pydot
!pip install graphviz

In [54]:
# plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [55]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_split=0.1,
)

Epoch 1/10
326/326 [==============================] - 206s 633ms/step - loss: 0.9101 - accuracy: 0.8752 - val_loss: 0.6324 - val_accuracy: 0.9020
Epoch 2/10
326/326 [==============================] - 197s 605ms/step - loss: 0.5209 - accuracy: 0.9162 - val_loss: 0.4900 - val_accuracy: 0.9221
Epoch 3/10
326/326 [==============================] - 195s 598ms/step - loss: 0.4200 - accuracy: 0.9304 - val_loss: 0.4341 - val_accuracy: 0.9296
Epoch 4/10
326/326 [==============================] - 196s 601ms/step - loss: 0.3648 - accuracy: 0.9386 - val_loss: 0.4029 - val_accuracy: 0.9343
Epoch 5/10
326/326 [==============================] - 196s 601ms/step - loss: 0.3286 - accuracy: 0.9444 - val_loss: 0.3881 - val_accuracy: 0.9369
Epoch 6/10
326/326 [==============================] - 196s 601ms/step - loss: 0.3013 - accuracy: 0.9487 - val_loss: 0.3775 - val_accuracy: 0.9385
Epoch 7/10
326/326 [==============================] - 195s 599ms/step - loss: 0.2792 - accuracy: 0.9528 - val_loss: 0.3705 -

In [56]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [57]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [58]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = embedding_layer_h(decoder_inputs_single)

In [59]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [60]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [61]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [62]:
# from keras.utils import plot_model
# plot_model(decoder_model, to_file='model_plot_dec.png', show_shapes=True, show_layer_names=True)

In [63]:
### predictions

In [64]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [65]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [66]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: blue kettle on th stove
Response: स्टोव के पर नीला स्टोव


In [67]:
test_sentences = input_val
test_sentences_hindi = target_val

In [68]:
print(len(test_sentences) , len(test_sentences_hindi))

5786 5786


In [69]:
test_sentences.index('')

3575

In [70]:
test_sentences.pop(3575) , test_sentences_hindi.pop(3575)

('', 'समुद्र तट पर बाइक')

In [71]:
print(len(test_sentences) , len(test_sentences_hindi))

5785 5785


In [72]:
test_sentences.pop(1346)
test_sentences_hindi.pop(1346)


''

In [73]:
test_sentences.pop(1655)
test_sentences_hindi.pop(1655)

''

In [74]:
print(len(test_sentences) , len(test_sentences_hindi))

5783 5783


In [75]:
test_sentences.pop(258)
test_sentences_hindi.pop(258)
print(len(test_sentences) , len(test_sentences_hindi))

5782 5782


In [76]:
test_tensor = [[eng_lang_tokenizer.word_index[s] for s in en.split(' ')] for en in test_sentences]
# print(max_length(test_tensor))
#pad
test_sequences = pad_sequences(test_tensor, maxlen=max_input_len, padding='pre')
print(max_input_len)
# encoder_input_sequences = pad_sequences(input_tensor, maxlen=max_length_inp,padding='pre')

24


In [77]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]
    print('-')
    print('Input:', test_sentences[i])
    print('Response:', translation)

    print('Original Hindi Sentence:' , hindi_test[0])
    print(type(hindi_test[0]))

    


-
Input: tan wicker basket in the corner of room
Response: कमरे के कोने में बड़ा समूह |
Original Hindi Sentence: कमरे के कोने में टैन विकर की टोकरी
<class 'str'>
-
Input: boys are playing football
Response: लड़के के साथ खेल रहे हैं
Original Hindi Sentence: लड़के फुटबॉल खेल रहे हैं
<class 'str'>
-
Input: part of a wall
Response: एक दीवार का हिस्सा
Original Hindi Sentence: एक दीवार का हिस्सा
<class 'str'>
-
Input: dark brick on wall
Response: दीवार पर ईंट के पीछे
Original Hindi Sentence: दीवार पर गहरी ईंट
<class 'str'>
-
Input: stop sign with symbols
Response: स्टॉप के साथ धातु स्टॉप साइन
Original Hindi Sentence: ईशारों के साथ रूकने का चिन्ह
<class 'str'>
-
Input: a man hanging on a lamppost
Response: एक आदमी एक पोल पर एक आदमी है
Original Hindi Sentence: एक आदमी एक लंगोटी पर लटका
<class 'str'>
-
Input: a window on the building
Response: इमारत पर एक खिड़की
Original Hindi Sentence: भवन पर एक खिड़की
<class 'str'>
-
Input: bun length hot dog
Response: गर्म कुत्ता हॉट डॉग
Original Hindi Sente

-
Input: this is a pick up
Response: यह एक दूसरे है
Original Hindi Sentence: यह एक पिक अप है
<class 'str'>
-
Input: kid wearing protective headgear
Response: बच्चा पहने हुए
Original Hindi Sentence: बच्चे सुरक्षात्मक टोपी पहने हुए
<class 'str'>
-
Input: a man riding a motorcycle
Response: एक आदमी मोटरसाइकिल की सवारी करता है
Original Hindi Sentence: एक आदमी मोटर साइकिल की सवारी कर रहा है
<class 'str'>
-
Input: part of a shirt
Response: एक शर्ट का हिस्सा
Original Hindi Sentence: एक शर्ट का हिस्सा
<class 'str'>
-
Input: one octagonal red and white sign
Response: एक लाल और सफेद चिह्न
Original Hindi Sentence: एक अष्टकोणीय लाल और सफेद संकेत
<class 'str'>
-
Input: desert bar in the hand
Response: हाथ में रैकेट का है
Original Hindi Sentence: हाथ में डेजर्ट बार
<class 'str'>
-
Input: a boy with a red hat
Response: लाल टोपी वाला लड़का
Original Hindi Sentence: लाल टोपी वाला लड़का
<class 'str'>
-
Input: snow on the ground
Response: जमीन पर बर्फ
Original Hindi Sentence: जमीन पर बर्फ
<class 'str'>
-


In [78]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]
#     print('-')
#     print('Input:', test_sentences[i])
#     print('Response:', translation)

#     print('Original Hindi Sentence:' , hindi_test[0])
#     print(type(hindi_test[0]))
    
#     गिराफ़ की पूंछ होती है
#     reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
#     candidate = ['this', 'is', 'a', 'test']
#     score = sentence_bleu(reference, candidate)
#     print(score)
    
    hindi_ref = hindi_test[0]
    hindi_ref = hindi_ref.split()
#     print(hindi_ref)
    candidate = translation.split()
#     print(candidate)
    score = sentence_bleu([hindi_ref], candidate, weights=(1, 0, 0, 0) )

#     print(score)
    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)
    

C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram

100
the average bleu score is  0.7011576333097754


In [79]:

from nltk.translate.bleu_score import sentence_bleu

count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]

    
    hindi_ref = hindi_test[0]
    hindi_ref = hindi_ref.split()

    candidate = translation.split()

    score = sentence_bleu([hindi_ref], candidate, weights=(0, 1, 0, 0) )


    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)

100
the average bleu score is  0.38222629411126924


In [80]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]

    
    hindi_ref = hindi_test[0]
    hindi_ref = hindi_ref.split()

    candidate = translation.split()

    score = sentence_bleu([hindi_ref], candidate, weights=(0.25, 0.25, 0.25, 0.25) )


    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)

100
the average bleu score is  0.19784265887769897
